# Paper calculations

In [1]:
import numpy as np
import astropy.units as u
import astropy.constants as ct
from astropy.modeling.models import BlackBody

# Infalling times

Free fall time of central region:

In [2]:
# ALMA1 measurements and source and observational parameters
d = 3.1 * u.kpc
Fcenter = 7.7 * u.mJy
rcenter = 121 / 2 * u.au
Tcenter = 100 * u.K
nu = 226.15 * u.GHz
kappa = 1.0 * u.cm**2/u.g
Rdg = 0.01
Mstar = 10 * u.M_sun

# Mass: central dust mass + source mass
bb = BlackBody(temperature=Tcenter)
Bnu = bb(nu).to(u.W/u.m**2/u.Hz, u.dimensionless_angles())
Mcenter = Fcenter * d**2 / (Rdg * kappa * Bnu)
Mcenter = Mcenter.to(u.M_sun)
Mtotal = Mcenter + Mstar
print('Central dust mass: ', Mcenter)
print('Central total mass: ', Mtotal)

# Free fall time
rho = Mtotal / (4/3 * np.pi * rcenter**3)
tff = np.sqrt(3 * np.pi / (32 * ct.G * rho))

# For disk https://wwwmpa.mpa-garching.mpg.de/~henk/pub/disksn.pdf
#tdyn = np.sqrt(r**3 / (ct.G * (mass + Mc)))

print('Free fall time:', tff.to(u.yr))
#print('Free fall time (disk):', tdyn.to(u.yr))

Central dust mass:  0.23819104430758484 solMass
Central total mass:  10.238191044307586 solMass
Free fall time: 25.998881260017974 yr


Viscous accretion

In [3]:
# Alpha disk parameters
alpha = np.array([0.1, 1])
vrot = np.sqrt(ct.G * Mstar / rcenter)
mu = 2.33
mH = 1.00784 * u.u
omega = vrot / rcenter
cs = np.sqrt(ct.k_B * Tcenter / (mu * mH))

tvis = rcenter**2 * omega / (3 * cs**2 * alpha)
print('Viscous accretion times at central source radius:', tvis.to(u.yr), 'for alpha ', alpha)

# For centrifugal radius
rcentrifugal = 500 * u.au
vrot_centrifugal = np.sqrt(ct.G * Mstar / rcentrifugal)
omega_centrifugal = vrot_centrifugal / rcentrifugal
tvis_centrifugal = rcentrifugal**2 * omega_centrifugal / (3 * cs**2 * alpha)
print('Viscous accretion times at centrifugal radius:', tvis_centrifugal.to(u.yr), 'for alpha ', alpha)

Viscous accretion times at central source radius: [32695.22386785  3269.52238678] yr for alpha  [0.1 1. ]
Viscous accretion times at centrifugal radius: [93992.16002864  9399.21600286] yr for alpha  [0.1 1. ]


Streamer masses:

In [4]:
# For both streamers
Tstream = np.array([100, 300, 500]) * u.K
bb = BlackBody(temperature=Tstream)
Bnu = bb(nu).to(u.W/u.m**2/u.Hz, u.dimensionless_angles())

# Blue streamer
Fblue = 20.7 * u.mJy
mass_blue = Fblue * d**2 / (Rdg * kappa * Bnu)
mass_blue = mass_blue.to(u.M_sun)

# Red streamer
Fred = 11.1 * u.mJy
mass_red = Fred * d**2 / (Rdg * kappa * Bnu)
mass_red = mass_red.to(u.M_sun)

print('Temperatures: ', Tstream)
print('Blue streamer mass: ', mass_blue)
print('Red streamer mass: ', mass_red)

Temperatures:  [100. 300. 500.] K
Blue streamer mass:  [0.64033177 0.20577012 0.12256769] solMass
Red streamer mass:  [0.34336631 0.1103405  0.0657247 ] solMass


Streamer times:

In [5]:
# Velocity
rc = 500 * u.au
vff = np.sqrt(2 * ct.G * Mstar / rc)
print('Free fall velocity: ', vff.to(u.km/u.s))

# Infall rates
length = rc
infall_blue = vff * mass_blue / length
infall_red = vff * mass_red / length
infall_blue = infall_blue.to(u.M_sun/u.yr)
infall_red = infall_red.to(u.M_sun/u.yr)
inf_visc_blue = (mass_blue[0]/tvis_centrifugal).to(u.M_sun/u.yr)
inf_visc_red = (mass_red[0]/tvis_centrifugal).to(u.M_sun/u.yr)
print('Infall rate blue streamer: ', infall_blue)
print('Infall rate red streamer: ', infall_red)
print('Infall rate for viscous accretion blue streamer (T=100K): ', inf_visc_blue, 'for alpha ', alpha)
print('Infall rate for viscous accretion red streamer (T=100K): ', inf_visc_red, 'for alpha ', alpha)

# To replenish the central region
trep = Mcenter / (infall_red + infall_blue)
trep_blue = Mcenter / infall_blue
trep_red = Mcenter / infall_red
trep_visc = Mcenter / (inf_visc_blue + inf_visc_red)
trep_visc_blue = Mcenter / inf_visc_blue
trep_visc_red = Mcenter / inf_visc_red
print('Replenish time total: ', trep)
print('Replenish time blue streamer: ', trep_blue)
print('Replenish time red streamer: ', trep_red)
print('Replenish time for viscous accretion total (T=100K): ', trep_visc, 'for alpha ', alpha)
print('Replenish time for viscous accretion blue streamer (T=100K): ', trep_visc_blue, 'for alpha ', alpha)
print('Replenish time for viscous accretion red streamer (T=100K): ', trep_visc_red, 'for alpha ', alpha)

Free fall velocity:  5.956938365935386 km / s
Infall rate blue streamer:  [0.0016093  0.00051715 0.00030804] solMass / yr
Infall rate red streamer:  [0.00086296 0.00027731 0.00016518] solMass / yr
Infall rate for viscous accretion blue streamer (T=100K):  [6.81260829e-06 6.81260829e-05] solMass / yr for alpha  [0.1 1. ]
Infall rate for viscous accretion red streamer (T=100K):  [3.65313778e-06 3.65313778e-05] solMass / yr for alpha  [0.1 1. ]
Replenish time total:  [ 96.34561379 299.81592256 503.33949364] yr
Replenish time blue streamer:  [148.00920379 460.58677959 773.24617864] yr
Replenish time red streamer:  [ 276.01716383  858.93210247 1441.99963044] yr
Replenish time for viscous accretion total (T=100K):  [22759.10793146  2275.91079315] yr for alpha  [0.1 1. ]
Replenish time for viscous accretion blue streamer (T=100K):  [34963.26725703  3496.3267257 ] yr for alpha  [0.1 1. ]
Replenish time for viscous accretion red streamer (T=100K):  [65201.76866852  6520.17686685] yr for alpha  

Streamer viscous times:

In [6]:
# Alpha disk parameters
vrot = np.sqrt(ct.G * Mstar / rc)
mu = 2.33
mH = 1.00784 * u.u
omega = vrot / rc
cs = np.sqrt(ct.k_B * Tstream / (mu * mH))

for alp in alpha:
    tvis = rc**2 * omega / (3 * cs**2 * alp)
    infall_red_vis = mass_red / tvis
    infall_blue_vis = mass_blue / tvis
    infall_blue_vis = infall_blue_vis.to(u.M_sun/u.yr)
    infall_red_vis = infall_red_vis.to(u.M_sun/u.yr)
    print('Streamer viscous accretion times:', tvis.to(u.yr), 'for alpha ', alp)
    print('Viscous infall rate blue streamer: ', infall_blue_vis)
    print('Viscous infall rate red streamer: ', infall_red_vis)

Streamer viscous accretion times: [93992.16002864 31330.72000955 18798.43200573] yr for alpha  0.1
Viscous infall rate blue streamer:  [6.81260829e-06 6.56767914e-06 6.52010159e-06] solMass / yr
Viscous infall rate red streamer:  [3.65313778e-06 3.52179896e-06 3.49628636e-06] solMass / yr
Streamer viscous accretion times: [9399.21600286 3133.07200095 1879.84320057] yr for alpha  1.0
Viscous infall rate blue streamer:  [6.81260829e-05 6.56767914e-05 6.52010159e-05] solMass / yr
Viscous infall rate red streamer:  [3.65313778e-05 3.52179896e-05 3.49628636e-05] solMass / yr


## Streamer momentum vs radiation pressure

Order of magnitude comparison (Keto & Wood 2006):

In [7]:
# Radiation
L = 10**4.396 * u.Lsun
force1 = L / ct.c
force1 = force1.to(u.N)

# Inflow
vel_impact = np.sqrt(2 *  ct.G * Mstar / rcenter)
force2_100 = infall_blue[0] * vel_impact
force2_100 = force2_100.to(u.N)

print('Luminosity:', L)
print('Inpact velocity:', vel_impact.to(u.km/u.s))
print('Radiation force:', force1)
print('Inflow force T=100K:', force2_100)


Luminosity: 24888.573182823904 solLum
Inpact velocity: 17.124993743270366 km / s
Radiation force: 3.1779804862152306e+22 N
Inflow force T=100K: 1.736477142156722e+24 N
